In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"]= (10,2)
from statsmodels.tsa.stattools import adfuller
import warnings
warnings.filterwarnings("ignore")
import plotly.express as px

In [ ]:
def sliderPlot(df1,t=None):
    if t =="line":
        fig = px.line(x=df1.index, y=df1)
    else:
        fig = px.scatter(x=df1.index, y=df1)
    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(count=2, label="2y", step="year", stepmode="backward"),
                dict(count=3, label="3y", step="year", stepmode="backward"),
                dict(step="all")
            ])))
    fig.show()
    return
def scatter(df1):
    plt.scatter(df1.index, df1, color='k',s=10)
    plt.xlabel('timestamp')
    # plt.title('current')
    plt.show()

In [2]:
""" data ingestion """

lst = ['wm',"ac1",'ac2',"f",'tv']
path = "d:/NILM/Data_files/Parquet/Appliances12_06-04_08/"

""" data ingestion done """

""" data validation and transformation """
complete_data_lst = [] # empty list to store final data set of all appliances
for i in range(len(lst)):
    df = pd.read_parquet(f"{path}{lst[i]}.parquet")
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df.drop_duplicates(inplace=True)  
    df.set_index(['timestamp'],inplace=True ,drop=True)
    df.drop(df[(df['frequency']>51) | (df['frequency']<49)].index, inplace=True)
    df.drop(df[(df['PF']>1) | (df['PF']<0)].index, inplace=True)
    # df.drop(df[(df['current']>20) |(df['current']<0.07)].index, inplace=True)
    df.drop(df[df['voltage']<140].index, inplace=True)
    
    """ data validation done """
    
    """ data transformation """
    # power is active power derived from voltage current and pf
    df['power'] = (df['voltage'] * df['current'] * df['PF']).round(2)
    
    if lst[i]=="f":
        df_appliance = df[['power',"voltage",'current']].resample(rule="1s").asfreq()
        df_appliance = df_appliance.rename(columns={'power': f'{lst[i]}_A', 'voltage': f'{lst[i]}_V', 'current': f'{lst[i]}_C'})
    else:
        df_sample_lst = [] # empty list for storing resampled data at 1sec frequency of each hour
        unique_dates = pd.Series(df.index.date).unique()
        for date1 in unique_dates: # dates when applince is on
            df_date = df[df.index.to_series().dt.date == date1] # dataframe of that perticular date
            hours = df_date.index.hour.unique() # hours when appliance is on
            for h in hours:
                date2 = f"{date1} {h:02d}"
                df_hour = df_date.loc[date2] # dataframe of that perticular hour at which appliance was on 
                df_resampled_hour = df_hour[['power',"voltage",'current']].resample(rule="1s").asfreq()
                df_sample_lst.append(df_resampled_hour) # adding each hour data in lst
        df_appliance = pd.concat(df_sample_lst) # concating all hours data into single data frame with missing values within signature
        df_appliance = df_appliance.rename(columns={'power': f'{lst[i]}_A', 'voltage': f'{lst[i]}_V', 'current': f'{lst[i]}_C'})
    complete_data_lst.append(df_appliance) # all applince data in list


In [10]:
for i in range(len(complete_data_lst)):
    df = complete_data_lst[i]
    null_values = df[f'{lst[i]}_V'].isna().sum()
    print(f"total values in {lst[i]}:",len(df))
    print(f"null values in {lst[i]}:",null_values)

total values in wm: 27663
null values in wm: 6778
total values in ac1: 347606
null values in ac1: 9874
total values in ac2: 460529
null values in ac2: 33996
total values in f: 4579201
null values in f: 535216
total values in tv: 539854
null values in tv: 125155


In [5]:
for i in range(len(complete_data_lst)):
    df = complete_data_lst[i]
    print("start", df.first_valid_index())
    print("end",df.last_valid_index())

start 2013-06-14 01:22:10
end 2013-07-31 02:50:39
start 2013-06-12 15:37:44
end 2013-08-03 17:32:20
start 2013-06-12 16:05:36
end 2013-08-03 18:48:57
start 2013-06-12 00:00:00
end 2013-08-04 00:00:00
start 2013-06-12 06:26:28
end 2013-08-03 14:55:31


In [3]:
final_df = pd.concat(complete_data_lst, axis=1)
final_df

,wm_A,wm_V,wm_C,ac1_A,ac1_V,ac1_C,ac2_A,ac2_V,ac2_C,f_A,f_V,f_C,tv_A,tv_V,tv_C
timestamp,,,,,,,,,,,,,,,
2013-06-12 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,127.53,240.02,0.595,NaN,NaN,NaN
2013-06-12 00:00:01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,127.01,240.11,0.593,NaN,NaN,NaN
2013-06-12 00:00:02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,126.81,240.14,0.592,NaN,NaN,NaN
2013-06-12 00:00:03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,126.22,240.10,0.590,NaN,NaN,NaN
2013-06-12 00:00:04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,125.85,240.07,0.589,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-08-03 23:59:56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79.93,235.81,0.420,NaN,NaN,NaN
2013-08-03 23:59:57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.09,235.77,0.422,NaN,NaN,NaN
2013-08-03 23:59:58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
